## Implementation of CNNs

###  Sequenctial Method

In [1]:
import torch
import torch.nn as nn
import numpy as np 

In [2]:
batch_size =32

N, n_C, n_H, n_W = 32,3,28,28

kernel_size = 3

padding_size = (kernel_size-1)//2

pool_size, pool_strides = 2, 2

n_conv_neurons = [10,20,30]   # for 문을 통해 Conv Lavyer 끼리 Dense Layer끼리 구현하는 것도 가능

n_dense_neurons = [50,30,10]

x = torch.rand((N, n_C, n_H, n_W))

model = nn.Sequential(
            nn.Conv2d(n_C, n_conv_neurons[0],kernel_size,padding = padding_size),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_strides ),
            nn.Conv2d(n_conv_neurons[0], n_conv_neurons[1],kernel_size, padding = padding_size),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_strides ),
            nn.Conv2d(n_conv_neurons[1], n_conv_neurons[2],kernel_size, padding = padding_size),
            nn.ReLU(),
            nn.MaxPool2d(pool_size,pool_strides ),
            nn.Flatten(),
            nn.Linear(270,n_dense_neurons[0]),
            nn.ReLU(),
            nn.Linear(n_dense_neurons[0],n_dense_neurons[1]),
            nn.ReLU(),
            nn.Linear(n_dense_neurons[1],n_dense_neurons[2]),
            nn.Softmax(dim=1)
    
)

prediction = model(x)
prediction.shape

torch.Size([32, 10])

## Model Class

In [3]:
batch_size =32

N, n_C, n_H, n_W = 32,3,28,28

kernel_size = 3

padding_size = (kernel_size-1)//2

pool_size, pool_strides = 2, 2

n_conv_neurons = [10,20,30]   # for 문을 통해 Conv Lavyer 끼리 Dense Layer끼리 구현하는 것도 가능

n_dense_neurons = [50,30,10]

x = torch.rand((N, n_C, n_H, n_W))

class TestCNN(nn.Module):
    def __init__(self):
        super(TestCNN, self).__init__()
        global n_C
        self.conv1      = nn.Conv2d(n_C, n_conv_neurons[0],kernel_size,padding = padding_size )
        self.conv2      = nn.Conv2d(n_conv_neurons[0], n_conv_neurons[1],kernel_size,padding = padding_size )
        self.conv3      = nn.Conv2d(n_conv_neurons[1], n_conv_neurons[2],kernel_size,padding = padding_size )
        self.activation = nn.ReLU()
        self.MaxPool    = nn.MaxPool2d(pool_size,pool_strides )
        self.flatten    = nn.Flatten()
        self.linear1    = nn.Linear(270,n_dense_neurons[0])
        self.linear2    = nn.Linear(n_dense_neurons[0],n_dense_neurons[1])
        self.linear3    = nn.Linear(n_dense_neurons[1],n_dense_neurons[2])
        self.softmax    = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        x = self.conv3(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation(x)
        print(x.shape)
        x = self.linear2(x)
        x = self.activation(x)
        print(x.shape)
        x = self.linear3(x)
        x = self.softmax(x)
        print(x.shape)
        return x
    
model = TestCNN()
prediction = model(x)

torch.Size([32, 10, 14, 14])
torch.Size([32, 20, 7, 7])
torch.Size([32, 30, 3, 3])
torch.Size([32, 50])
torch.Size([32, 30])
torch.Size([32, 10])


## Sequential + Layer Sub-classing

In [4]:
batch_size =32

N, n_C, n_H, n_W = 32,3,28,28

kernel_size = 3

padding_size = (kernel_size-1)//2

pool_size, pool_strides = 2, 2

n_chnnals = [3,10,20,30]   # for 문을 통해 Conv Lavyer 끼리 Dense Layer끼리 구현하는 것도 가능

n_dense_neurons = [50,30,10]

x = torch.rand((N, n_C, n_H, n_W))

class MyConv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MyConv, self).__init__()
        self.conv       = nn.Conv2d(in_channel, out_channel, kernel_size, padding = (kernel_size -1 )//2)
        self.activation = nn.ReLU()
        self.MaxPool    = nn.MaxPool2d(pool_size,pool_strides )
    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        return x
    
model = nn.Sequential(
        MyConv(n_chnnals[0],n_chnnals[1]),
        MyConv(n_chnnals[1],n_chnnals[2]),
        MyConv(n_chnnals[2],n_chnnals[3]),
        nn.Flatten(),
        nn.Linear(270,n_dense_neurons[0]),
        nn.Linear(n_dense_neurons[0],n_dense_neurons[1]),
        nn.Linear(n_dense_neurons[1],n_dense_neurons[2]),
        nn.Softmax(dim=1)
)
prediction = model(x)
print(prediction.shape)

torch.Size([32, 10, 14, 14])
torch.Size([32, 20, 7, 7])
torch.Size([32, 30, 3, 3])
torch.Size([32, 10])


## Sequential + Model Sub-classing

In [7]:
batch_size =32

N, n_C, n_H, n_W = 32,3,28,28

kernel_size = 3

padding_size = (kernel_size-1)//2

pool_size, pool_strides = 2, 2

n_chnnals = [3,10,20,30]   # for 문을 통해 Conv Lavyer 끼리 Dense Layer끼리 구현하는 것도 가능

n_dense_neurons = [50,30,10]

x = torch.rand((N, n_C, n_H, n_W))


class MyConv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MyConv, self).__init__()
        self.conv       = nn.Conv2d(in_channel, out_channel, kernel_size, padding = (kernel_size -1 )//2)
        self.activation = nn.ReLU()
        self.MaxPool    = nn.MaxPool2d(pool_size,pool_strides )
    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        return x

class TestCNN(nn.Module):
    def __init__(self):
        super(TestCNN, self).__init__()
        self.conv1      = MyConv(n_chnnals[0],n_chnnals[1])
        self.conv2      = MyConv(n_chnnals[1],n_chnnals[2])
        self.conv3      = MyConv(n_chnnals[2],n_chnnals[3])
        self.activation = nn.ReLU()
        self.flatten    = nn.Flatten()
        self.linear1    = nn.Linear(270,n_dense_neurons[0])
        self.linear2    = nn.Linear(n_dense_neurons[0],n_dense_neurons[1])
        self.linear3    = nn.Linear(n_dense_neurons[1],n_dense_neurons[2])
        self.softmax    = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.conv3(x)
        print(x.shape)
        
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation(x)
        print(x.shape)
        x = self.linear2(x)
        x = self.activation(x)
        print(x.shape)
        x = self.linear3(x)
        x = self.softmax(x)
        print(x.shape)
        return x

model = TestCNN()
prediction = model(x)
print(prediction.shape)

torch.Size([32, 10, 14, 14])
torch.Size([32, 10, 14, 14])
torch.Size([32, 20, 7, 7])
torch.Size([32, 20, 7, 7])
torch.Size([32, 30, 3, 3])
torch.Size([32, 30, 3, 3])
torch.Size([32, 50])
torch.Size([32, 30])
torch.Size([32, 10])
torch.Size([32, 10])


In [12]:
batch_size =32

N, n_C, n_H, n_W = 32,3,28,28

kernel_size = 3

padding_size = (kernel_size-1)//2

pool_size, pool_strides = 2, 2

n_chnnals = [3,10,20,30]   # for 문을 통해 Conv Lavyer 끼리 Dense Layer끼리 구현하는 것도 가능

n_dense_neurons = [50,30,10]

x = torch.rand((N, n_C, n_H, n_W))


class MyConv(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(MyConv, self).__init__()
        self.conv       = nn.Conv2d(in_channel, out_channel, kernel_size, padding = (kernel_size -1 )//2)
        self.activation = nn.ReLU()
        self.MaxPool    = nn.MaxPool2d(pool_size,pool_strides )
    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        x = self.MaxPool(x)
        print(x.shape)
        return x

class TestCNN(nn.Module):
    def __init__(self):
        super(TestCNN, self).__init__()
        self.fe        =nn.Sequential()
        self.fe.add_module("conv_1",MyConv(n_chnnals[0],n_chnnals[1]))
        self.fe.add_module("conv_2",MyConv(n_chnnals[1],n_chnnals[2]))
        self.fe.add_module("conv_3",MyConv(n_chnnals[2],n_chnnals[3]))
        
        self.classifier = nn.Sequential()
        self.classifier.add_module('flatten',nn.Flatten() )
        self.classifier.add_module('Dense1' ,nn.Linear(270,n_dense_neurons[0]) )
        self.classifier.add_module('relu'   ,nn.ReLU() )
        self.classifier.add_module('Dense2' ,nn.Linear(n_dense_neurons[0],n_dense_neurons[1]) )
        self.classifier.add_module('Dense3' ,nn.Linear(n_dense_neurons[1],n_dense_neurons[2]) )
        self.classifier.add_module('softmax',nn.Softmax(dim=1) )
    
    def forward(self, x):
        x = self.fe(x)
        print(x.shape)
        
        x = self.classifier(x)
        print(x.shape)
        return x

model = TestCNN()
prediction = model(x)

torch.Size([32, 10, 14, 14])
torch.Size([32, 20, 7, 7])
torch.Size([32, 30, 3, 3])
torch.Size([32, 30, 3, 3])
torch.Size([32, 10])
